In [1]:
from att_visual import generate
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
from pytorch_transformers import modeling_bert
import numpy as np
import torch
from pytorch_transformers import BertTokenizer
import pdb
import torch.nn.functional as F
import sys
model_name = 'Complete_Bert'
import time
import utils
import imp
import os
from sklearn.metrics import confusion_matrix
import json

In [3]:
imp.reload(utils)
hpara1 = utils.hpara()

In [4]:
hpara1.ratio = 2 
hpara1.batch_size = 1
hpara1.word_layers=1
hpara1.sent_layers=1
hpara1.use_position_embedding = True
hpara1.use_angular = False

In [5]:
imp.reload(utils)
training_generator,validation_generator,dummy_generator = utils.load_data(ratio=hpara1.ratio,clarity =True)
tokenizer = utils._load_tf_tokenizer(vocab_file = '/gpfs/qlong/home/tzzhang/nlp_test/bert/mimic_based_complete_model/vocab.txt')

In [6]:
# Set up the word embedding model but do not load the pre-trained parameters
tf_src_dir = '/gpfs/qlong/home/tzzhang/nlp_test/bert/mimic_based_complete_model'
bert_config_file = os.path.join(tf_src_dir,'bert_config.json')
config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)

In [7]:
config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
use_advanced_loss = hpara1.use_angular
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,use_advanced_loss=use_advanced_loss,use_position_embedding=use_position_embedding)

In [37]:
model_dir = '/gpfs/qlong/home/tzzhang/mimicIII/nn_code/exp/Complete_Bert_2019_11_29_1layer_Emb'
i = 8
word_model_path = os.path.join(model_dir,'save_word_'+str(i) + '.bin')
sent_model_path = os.path.join(model_dir,'save_sent_'+str(i) + '.bin')
model_word.load_state_dict(torch.load(word_model_path))
model_sent.load_state_dict(torch.load(sent_model_path))

<All keys matched successfully>

In [38]:
model_word = model_word.cuda()
model_sent = model_sent.cuda()

In [39]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './exp/'+model_name+'_'+date[1:-1]+'_visual/'
if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
imp.reload(utils)
aaa = utils.output_att_scores(hpara1,model_word,model_sent,save_dir,training_generator,validation_generator,use_angular=False)

In [40]:
type(aaa)

generator

In [41]:
word_att_list,sent_att_score,doc,proba = next(aaa)

In [42]:
proba

tensor([[-1.9376, -0.1555]], device='cuda:0', grad_fn=<LogSoftmaxBackward>)

In [13]:
type(qqq)

list

In [14]:
len(qqq)

227

In [22]:
type(qqq[0])

tuple

In [15]:
type(qqq[0])

tuple

In [16]:
len(qqq[0])

1

In [19]:
qqq[0][0].shape

torch.Size([1, 12, 64, 64])

In [21]:
len(www)

1

In [14]:
www[0].shape

NameError: name 'www' is not defined

In [16]:
from scipy.special import softmax
def One_layer_back_trace(att_heads): # need a tensor as input
    seq_len = att_heads.shape[2]
    layer_num = len(att_heads)
    last_layer_portion = np.diag([1] * seq_len)
    one_layer_att_heads = att_heads[0]
    sum_through_heads = torch.sum(one_layer_att_heads,0)
    sum_through_heads = sum_through_heads.cpu().detach().numpy()
    sum_through_heads = np.where(sum_through_heads > 0.001, sum_through_heads, sum_through_heads -1000)
    #pdb.set_trace()
    #prop_through_heads = sum_through_heads
    prop_through_heads = softmax(sum_through_heads,axis=1)
    curr_layer_portion = np.matmul(prop_through_heads, last_layer_portion)
    return curr_layer_portion

In [43]:
sent_weight = One_layer_back_trace(sent_att_score[0])

In [44]:
sent_weight[0,:]

array([0.00526846, 0.00385837, 0.00385744, 0.00386923, 0.00385646,
       0.00388383, 0.00385661, 0.00391922, 0.00386636, 0.00385651,
       0.00390121, 0.00385628, 0.00385411, 0.0038607 , 0.00385517,
       0.00385591, 0.00385591, 0.00385615, 0.00385653, 0.00596919,
       0.00385743, 0.00385301, 0.00385512, 0.00385553, 0.00385587,
       0.00385743, 0.00421096, 0.00385566, 0.00385814, 0.00448316,
       0.00404915, 0.00397496, 0.00394144, 0.00407067, 0.00387124,
       0.00386985, 0.00388103, 0.00386759, 0.00386959, 0.0038641 ,
       0.00386012, 0.00385714, 0.00385855, 0.0038798 , 0.00386788,
       0.00387206, 0.00385584, 0.00385786, 0.00385459, 0.00385569,
       0.00385747, 0.00394416, 0.00386365, 0.00387282, 0.00386524,
       0.0038546 , 0.0047695 , 0.00385683, 0.00385395, 0.0038576 ,
       0.00385881, 0.00385403, 0.00385556, 0.00386257, 0.00385518,
       0.00386358, 0.00385873, 0.00385383, 0.00386285, 0.00385805,
       0.00385548, 0.00385508, 0.00385832, 0.00385466, 0.00386

In [45]:
print(np.max(sent_weight[0,:]))
print(np.argmax(sent_weight[0,:]))

0.05804523825645447
228


In [50]:
big_factor = np.argpartition(sent_weight[0,:],-50)[-50:]

In [51]:
big_factor

array([136,  88,  86,  85,  90,  83,  97,  99, 102, 103,  77,  75, 129,
       131, 133, 134, 137, 140, 141,  56, 145, 153,   7,  51, 165, 167,
       168, 171, 172, 176, 179, 182, 185, 200, 202,  33,  32,  31,  30,
        29, 212,  26,  10,   0,  19, 135, 156, 157, 214, 228])

In [35]:
doc[56]

('L AKA, no edema of RLE, 2+ DP/PT pulses; quarter sized sacral ulcer with mild surrounding tenderness Pertinent Results:09:15AM',)

In [48]:
sent_weight[0,56]

0.00476949755102396

In [69]:
for i in big_factor:
    print(doc[i-1])

('Urine culture grew E coli and Klebsiella, sensitive to meropenem and bactrim.',)
('| AMIKACIN',)
('MIC expressed in MCG/ML _ _',)
('SENSITIVITIES:',)
('32',)
('KLEBSIELLA PNEUMONIAE.',)
('<=1 S CEFUROXIME 4 S 16',)
('4 R =',)
('16 R MEROPENEM <=0.25 S <=0.25 S NITROFURANTOIN <',)
('=16 S 128 R PIPERACILLIN/TAZO <=4 S 32',)
('RBC-',)
('NITRITE- NEG PROTEIN-30 GLUCOSE-NEG KETONE-NEG BILIRUBIN-NEG UROBILNGN-NEG PH-6.0',)
('Surgical staples projecting over the left upper and lower quadrants remain similar in appearance as well.',)
('Patient was initially febrile to 101.8 and hypotensive to 90/68 on admission.',)
('Foley was placed with purulent urine return.',)
('She was initially continued on meropenem and linezolid, but the linezolid was switched to daptomycin ondue to concern for serotonin syndrome given venlafaxine use.',)
('Due to this, meropenem was switched to bactrim onto complete a 14 day course.',)
('She was restarted on lisinopril and metoprolol XL after 24hrs w/o hypotension.

In [25]:
print(len(doc))
print(sent_att_score[0].)

[('Admission Date:',),
 ('Discharge Date:Service: MEDICINE Allergies:',),
 ('Patient recorded as having No Known Allergies to Drugs',),
 ('Attending:',),
 ('Chief Complaint:',),
 ('Fever, rigors Major Surgical or Invasive Procedure: None.',),
 ('History of Present Illness:',),
 ('Ms.is an 89yo spanish speaking female with PMH significant for multiple ESBL UTIs, lumbar osteomyelitis, and psoas abcess who was admitted to the MICU with a UTI and transient hypotension concerning for urosepsis.',),
 ('Of note, the patient was recently hospitalized infor E coli and K. pneumo UTI (BCx + for K. pneumo), tx with cephalexin/cefpodoxime for 14d.',),
 ('Her osteomyelitis was unchanged per MRI at this time.',),
 ("In addition, a PICC was placed onto administer a 7d course of Imipenem (500mg q8h,) for an ESBL E. coli UTIfoley catheter (d/c'd).",),
 ('The PICC was kept until f/u urine studies could be performed 1 wk post-abx.',),
 ("This morning at the patient's NH, she was noted to have fevers and r

In [57]:
sent_att_score_np = sent_att_score[0].detach().cpu().numpy()

In [68]:
np.sum(np.sum(sent_att_score_np[0,0],axis=0) > 0)

229

In [67]:
len(doc)

228

In [61]:
np.shape(sent_att_score_np[0,0])

(400, 400)

In [64]:
np.sum(sent_att_score_np[0,0],axis=0)

array([ 5.2361536,  1.3177215,  1.3547724,  1.3214734,  1.2952523,
        1.7645105,  1.3146765,  1.9620658,  1.5141255,  1.3498455,
        1.8282733,  1.3286585,  1.3176315,  1.307837 ,  1.3181812,
        1.3750317,  1.3011062,  1.2829152,  1.2961886,  4.7984095,
        1.3913524,  1.3171492,  1.2966104,  1.3152673,  1.321562 ,
        1.3092753,  2.9747598,  1.2996969,  1.3013085,  3.5189843,
        2.5863   ,  2.2988865,  2.0996196,  2.6259835,  1.6334065,
        1.5586624,  1.667915 ,  1.3660314,  1.3409575,  1.3365104,
        1.3000331,  1.3116885,  1.4323014,  1.3838404,  1.3130796,
        1.3511999,  1.3047053,  1.3139619,  1.344188 ,  1.3543758,
        1.2967087,  2.1267042,  1.3199633,  1.6463683,  1.3426415,
        1.3228385,  3.6382887,  1.2901404,  1.3103247,  1.3118771,
        1.3011801,  1.3056779,  1.362177 ,  1.2986312,  1.3219962,
        1.3137388,  1.2847735,  1.2959813,  1.2961322,  1.2959374,
        1.3610611,  1.338287 ,  1.4267422,  1.2914001,  1.5744

In [63]:
sent_att_score_np[0,0]

array([[2.2710731e-02, 6.5857212e-06, 1.0847792e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.2077434e-02, 2.2974026e-03, 2.4671985e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.9090483e-02, 2.4260636e-03, 2.5850176e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [7.0094955e-03, 3.4584000e-03, 3.5679159e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0512823e-02, 2.7541451e-03, 3.2008043e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [6.7653591e-03, 3.8588368e-03, 3.6692291e-03, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)